# Sandbox & premiers traitements sur le fichier "expenses"

## Chargement du fichier

In [1]:
import pandas as pd

df = pd.read_excel('1_Travel expanses.xlsx', dtype='object')

## Traitements

### Exploitation du pattern trouvé dans "Vendor Comment"

In [2]:
df.loc[df['Vendor Comment'].str.contains(':',regex=False, na=False), 'Contains State Code'] = 'exist'
df.loc[~(df['Vendor Comment'].str.contains(':',regex=False, na=False)), 'Contains State Code'] = 'not exist'

### Modifications de la date sur les quelques lignes le nécessitant

In [3]:
df.loc[9337, 'GL Date'] = '4/11/2015'

df.loc[11093, 'GL Date'] = '3/04/15'
df.loc[11094, 'GL Date'] = '11/12/15'
df.loc[11095, 'GL Date'] = '4/20/15'

df['GL Date'] = df['GL Date'].astype("Datetime64")

### Ajout de nouvelles dimensions

In [4]:
df.loc[df['Contains State Code'] == 'exist', 'State Code'] = df['Vendor Comment'].str.extract(r':([^;]*),', expand=False)

df['Quarter'] = pd.to_datetime(df['GL Date']).dt.quarter

df['Day Name'] = df['GL Date'].dt.day_name()

### Préparations finales avant export

In [5]:
df = df.fillna('')

cols = ['GL Date', 
        'Expense Category', 
        'Employee Name', 
        'Vendor Name',
        'Vendor Comment', 
        'Activity .', 
        'Activity Short Description',
        'Expense Description (Flex field)', 
        'Ferc Account', 
        'Amount - gross',
        'Amount - OTP share', 
        'Amount - MN share', 
        'Contains State Code',
        'State Code', 
        'Quarter',
        'Day Name']

new_cols = ['Date',
            'ExpenseCategory',
            'EmployeeName',
            'VendorName',
            'VendorComment',
            'ActivityCode',
            'ActivityLabel',
            'ExpenseDescription',
            'FercAccount',
            'AmountGross',
            'AmountOTP',
            'AmountMN',
            'ContainsStateCode',
            'StateCode',
            'Quarter',
            'DayName']

df.columns = new_cols

In [6]:
df.to_csv('expenses.csv', index=False)